In [1]:
#a)

import re 
import numpy as np
import scipy

#parsing
with open("18.txt") as f:
    lines = f.readlines()

#contains the coordinates of all lavabits
lavabits = []
for l in lines:
    lavabits.append([int(re.sub("[^0-9]", "",e)) for e in l.split(sep=",")])

#move all bits to avoid border problems 
lavabits = np.array(lavabits)+np.array([1,1,1])

#create grid, 0: empty, 1: lava bit
max = np.max(lavabits,axis=0)
room = np.zeros(shape=max+np.array([2,2,2]))
for p in lavabits:
    room[p[0],p[1],p[2]] = 1

#use convolution to compute the surface area (+6 if the gridpoint is a lava bit. -1 for each facing lavabit).
conv_kernel = np.array([
    [[0,0,0],
    [0,-1,0],
    [0,0,0]],
    [[0,-1,0],
    [-1,6,-1],
    [0,-1,0]],
    [[0,0,0],
    [0,-1,0],
    [0,0,0]]
])
a = scipy.ndimage.convolve(room, conv_kernel)
a = a*(a>0)
np.sum(a)

3564.0

In [19]:
#b)

#grid of same size as room, 1: air can flow to this grid point, 0: it can not
flow_room = np.zeros(shape=room.shape)
flow_room[:,:,0] = 1
flow_room[:,:,-1] = 1
flow_room[:,0,:] = 1
flow_room[:,-1] = 1
flow_room[0,:,:] = 1
flow_room[-1,:,:] = 1


flow_kernel = np.array([
    [[0,0,0],
    [0,1,0],
    [0,0,0]],
    [[0,1,0],
    [1,1,1],
    [0,1,0]],
    [[0,0,0],
    [0,1,0],
    [0,0,0]]
])

#complement room
anti_room = np.ones(shape=room.shape)-room

#continiously let the air flow until there is no change anymore
while True:
    flow_room_ = scipy.ndimage.convolve(flow_room, flow_kernel)
    flow_room_ = flow_room_ * anti_room
    flow_room_ = flow_room_ > 0
    
    if np.array_equal(flow_room_,flow_room):
        break

    flow_room = flow_room_

#now the lava surface is calculate by couting the the touching airflow bits for all lavabits
s = 0
for p in lavabits:
    s += np.sum(flow_kernel*flow_room[p[0]-1:p[0]+2,p[1]-1:p[1]+2,p[2]-1:p[2]+2])
s
